In [ ]:
from models import SegNet
import VipsML

base='/mnt/z9-slide-storage/VipsML'

In [ ]:
# Original image. Any VIPS-supported format of any size.
origs=[base+'/00-orig.tif']
# Annotated dataset. One channel, one colour per class (incremental from 0).
# Any VIPS-supported format, size same as original.
masks=[base+'/00-mask.tif']

# Larger padding, more overlap, and more training from one dataset.
# meta_ratio also gives a snapshot at a lower resolution of neighboring tissue.
images = [VipsML.VipsSegmentationML(orig,mask,frame_size=196, padding=30, meta_ratio=3) 
                              for orig, mask in zip(origs,masks)]

# Generator takes a list of VipsML-images and wraps a supply chain for the  
# fit_generator around it.
generator = VipsML.VipsGroupML(images, batch_size=20)

# To get sensible validation data, split dataset, keeping 10% for validation.
# Re-shuffled after each epoch.
trainer, validator = generator.split_generators(0.1)

# Load and compile model
model = SegNet(generator.input_shape, generator.classes)
model.compile(loss="categorical_crossentropy", optimizer='sgd')

# Fit model
model.fit_generator(trainer,validation_data=validator,epochs=200)